Importing Liabraries

In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.utils import resample
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

Data Reading

In [55]:
# Define the file path for the Excel file stored in Google Drive
# file_path = 'DataSet_EU_3k_5k.xlsx'
file_path = 'DataSet_EU_3k_5k.xlsx'

# Use pandas to read the Excel file located at the specified file path
data = pd.read_excel(file_path)



In [56]:
# Display the first few rows of the dataframe to get an overview of the data
data.head()

,Power_1,Power_2,Power_3,Power_4,Power_5,Power_6,Power_7,Power_8,Power_9,Power_10,...,GSNR_69,GSNR_70,GSNR_71,GSNR_72,GSNR_73,GSNR_74,GSNR_75,GSNR_76,No. Spans,Total Distance(m)
0,0.000000,0.000000,0.000007,0.000007,0.000007,0.000000,0.000007,0.0,0.000007,0.0,...,90.061284,0.000000,0.000000,0.000000,0.000000,95.045789,0.000000,0.000000,8,690608
1,0.000000,0.000007,0.000000,0.000007,0.000000,0.000007,0.000000,0.0,0.000000,0.0,...,0.000000,92.560867,0.000000,0.000000,89.544975,0.000000,0.000000,0.000000,8,690608
2,0.000000,0.000000,0.000000,0.000007,0.000000,0.000000,0.000000,0.0,0.000007,0.0,...,0.000000,0.000000,90.991977,0.000000,0.000000,0.000000,85.512399,83.114113,8,690608
3,0.000007,0.000007,0.000007,0.000007,0.000007,0.000000,0.000000,0.0,0.000007,0.0,...,76.409583,73.664915,74.651700,79.071217,0.000000,0.000000,0.000000,0.000000,8,690608
4,0.000000,0.000000,0.000007,0.000000,0.000007,0.000000,0.000000,0.0,0.000007,0.0,...,0.000000,0.000000,78.299438,74.910765,72.030907,73.790264,81.190439,0.000000,8,690608


In [57]:
# Define the attribute columns and the target variable
attribute_columns = [f'Power_{i}' for i in range(1, 77)] + [f'ASE_{i}' for i in range(1, 77)] + [f'NLI_{i}' for i in range(1, 77)]+ ['No. Spans'] + ['Total Distance(m)']

# Label
target_column = 'GSNR_1'

In [63]:
# MinMax Normalization
# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Normalize the attribute columns using MinMaxScaler and convert the result back to a DataFrame
x = scaler.fit_transform(data[attribute_columns])
x = pd.DataFrame(x, columns=attribute_columns)

# Extract the target variable values and reshape them for normalization
y = data[target_column].values.reshape(-1, 1)
# Normalize the target variable using MinMaxScaler and convert the result back to a DataFrame
y = scaler.fit_transform(y)
y = pd.DataFrame(y, columns=[target_column])

# Display the first few rows of the normalized data for selected columns
print("MinMax Normalized Data:")
print(x['Power_1'].head(), x['ASE_1'].head(), x['NLI_1'].head(), x['No. Spans'].head(), x['Total Distance(m)'].head())
print(y.head())

MinMax Normalized Data:
0    0.000000
1    0.000000
2    0.000000
3    0.973215
4    0.000000
Name: Power_1, dtype: float64 0    1.000000
1    0.974493
2    0.982290
3    0.000000
4    0.952433
Name: ASE_1, dtype: float64 0    0.0
1    0.0
2    0.0
3    1.0
4    0.0
Name: NLI_1, dtype: float64 0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: No. Spans, dtype: float64 0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: Total Distance(m), dtype: float64
     GSNR_1
0  0.000000
1  0.000000
2  0.000000
3  0.837358
4  0.000000


In [64]:
from sklearn.model_selection import train_test_split
# Define the function to split data according to the specified pattern
def custom_train_test_split(data, labels, samples_per_block=3000, train_samples_per_block=2500, test_samples_per_block=500):
    # Initialize lists to hold the training and testing data and labels
    train_data = []
    test_data = []
    train_labels = []
    test_labels = []

    # Determine the total number of samples in the data
    total_samples = len(data)

    # Loop over the data in blocks of the specified size (samples_per_block)
    for start in range(0, total_samples, samples_per_block):
        # Define the end of the current block
        end = start + samples_per_block
        # Define the end of the training samples within the current block
        train_end = start + train_samples_per_block

        # Adjust train_end and end if they exceed the total number of samples
        if train_end > total_samples:
            train_end = total_samples
        if end > total_samples:
            end = total_samples

        # Append the training and testing data and labels for the current block to the respective lists
        train_data.append(data[start:train_end])
        test_data.append(data[train_end:end])
        train_labels.append(labels[start:train_end])
        test_labels.append(labels[train_end:end])

    # Concatenate the lists into DataFrames and reset the index
    train_data = pd.concat(train_data).reset_index(drop=True)
    test_data = pd.concat(test_data).reset_index(drop=True)
    train_labels = pd.concat(train_labels).reset_index(drop=True)
    test_labels = pd.concat(test_labels).reset_index(drop=True)

    # Return the training and testing data and labels
    return train_data, test_data, train_labels, test_labels

# Create a list of attribute columns excluding those that start with 'frequency'
# attribute_columns_without_frequency = [col for col in attribute_columns if not col.startswith('frequency')]

# Assuming 'x' and 'y' are your DataFrames for MinMax normalized data
# Perform the custom train-test split on the normalized data
# X_train, X_test, Y_train, Y_test = custom_train_test_split(x[attribute_columns_without_frequency], y[target_column])
X_train, X_test, Y_train, Y_test = train_test_split(
    x[attribute_columns], 
    y[target_column], 
    test_size=0.2,  # Equivalent to test_samples_per_block / samples_per_block
    random_state=42  # For reproducibility
)


# Display shapes to verify the split
print("Normalized data with equal path distribution in train and test data :\n")
print("Train data shape:", X_train.shape)
print("Test data shape:", X_test.shape)
print("Train labels shape:", Y_train.shape)
print("Test labels shape:", Y_test.shape)

Normalized data with equal path distribution in train and test data :

Train data shape: (8, 230)
Test data shape: (2, 230)
Train labels shape: (8,)
Test labels shape: (2,)


In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pickle
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

# Load a dataset from Hugging Face
dataset = load_dataset('wasifis/400')

# Filter out rows with null values
def filter_nulls(example):
    return all(example[column] is not None for column in ['instruction', 'input', 'output'])

dataset = dataset.filter(filter_nulls)

# Convert the dataset to a pandas DataFrame
df = pd.DataFrame(dataset['train'])

# Define the attribute columns and the target variable
attribute_columns = ['input', 'instruction']
target_column = 'output'

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

# Tokenize the attribute and target columns
def tokenize_function(examples):
    return tokenizer(examples['input'], examples['instruction'], examples['output'], truncation=True, padding='max_length')

tokenized_data = df.apply(lambda row: tokenize_function(row), axis=1)

# Convert tokenized data to DataFrame
tokenized_df = pd.DataFrame(tokenized_data.tolist())

# Pad the tokenized sequences
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="np")
padded_data = data_collator(tokenized_df.to_dict(orient='records'))

# Extract the padded sequences
input_ids = padded_data['input_ids']
attention_mask = padded_data['attention_mask']
labels = padded_data['labels']

# MinMax Normalization
scaler = MinMaxScaler()

# Normalize the padded sequences using MinMaxScaler
x = scaler.fit_transform(input_ids)
x = pd.DataFrame(x)

# Normalize the target variable using MinMaxScaler
y = scaler.fit_transform(labels)
y = pd.DataFrame(y)

# Display the first few rows of the normalized data for selected columns
print("MinMax Normalized Data:")
print(x.head())
print(y.head())

# Split the dataset into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(
    x, 
    y, 
    test_size=0.2,  # Equivalent to test_samples_per_block / samples_per_block
    random_state=42  # For reproducibility
)

# Display shapes to verify the split
print("Normalized data with equal path distribution in train and test data :\n")
print("Train data shape:", X_train.shape)
print("Test data shape:", X_test.shape)
print("Train labels shape:", Y_train.shape)
print("Test labels shape:", Y_test.shape)

# Save the data to a file
with open('data.pkl', 'wb') as f:
    pickle.dump((X_train, Y_train, X_test, Y_test), f)

C:\Users\A.C\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


MinMax Normalized Data:
   0         1         2         3         4         5         6         7    \
0  0.0  0.001137  0.193446  0.964499  0.209394  0.098348  0.037375  0.035754   
1  0.0  0.001137  0.115766  0.000037  0.044014  0.000186  0.047866  0.072845   
2  0.0  0.001137  0.066578  0.000037  0.044014  0.000186  0.047866  0.072845   
3  0.0  0.001137  0.197650  0.000037  0.044014  0.000186  0.047866  0.072845   
4  0.0  0.001137  0.117886  0.000037  0.044014  0.000186  0.047866  0.083102   

        8         9    ...  502  503  504  505  506  507  508  509  510  511  
0  0.001109  0.074251  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1  0.193736  0.036502  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2  0.612196  0.036502  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3  0.193736  0.036722  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4  0.043895  0.038042  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 512 column